In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [29]:
def subtract_sources(residuals, sources, detector_name, echan, day, data_type = 'ctime'):
    """This function calculates the chi-square fit to the data of a given detector, day and energy channel and saves the figure in the appropriate folder\n
    Input:\n
    calculate.subtract_sources(residuals, sources (array of the form [[ra1, dec1], [ra2, dec2]]), detector_name, echan, day = YYMMDD, data_type = 'ctime')\n
    Output:\n
    0 = sub_residuals\n
    1 = fit_curve\n
    2 = sources_ang_bin\n
    3 = sources_plots\n
    4 = sub_coeffs"""
    
    #get the downtimes and plotting time from this day
    if data_type == 'ctime':
        ctime_data = readfile.ctime(readfile(), detector_name, day)
        echan_rate = ctime_data[4]
        bin_time = ctime_data[5]
        
        bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)
        total_rate = np.sum(echan_rate[1:-2], axis = 0)
    
    elif data_type == 'cspec':
        cspec_data = readfile.cspec(readfile(), detector_name, day)
        echan_rate = cspec_data[4]
        bin_time = cspec_data[5]

        bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)
        total_rate = np.sum(echan_rate[1:-2], axis = 0)
    
    else:
        print 'Invalid data type: ' + str(data_type) + '\n Please insert an appropriate data type (ctime or cspec).'
    
    
    
    if sources[0].shape == ():
        number = 1
    else:
        number = len(sources)
    
    

    pre_coeffs = [-1.]
    
    if number == 1:
        source_ra = sources[0]
        source_dec = sources[1]
    else:
        source_ra = sources[0,0]
        source_dec = sources[0,1]
    
    source_data = calc.burst_ang_bin(detector_name, day, source_ra, source_dec, bin_time_mid, data_type)
    source_ang_bin = source_data[0]
    source_ang_bin = calc.ang_eff(source_ang_bin, echan, data_type)[0]
    source_occ = calc.src_occultation_bin(day, source_ra, source_dec, bin_time_mid, detector_name, data_type)[0]
    
    source_ang_bin[np.where(source_occ == 0)] = 0
    source_ang_bin[np.where(total_rate == 0)] = 0
        
    source_ang_bin[source_ang_bin>0] = source_ang_bin[source_ang_bin>0] - np.min(source_ang_bin[source_ang_bin>0])
        
    sources_ang_bin = np.array([source_ang_bin])
    
    if number > 1:
        for i in range(1, number):
            source_ra = sources[i,0]
            source_dec = sources[i,1]
            
            source_data = calc.burst_ang_bin(detector_name, day, source_ra, source_dec, bin_time_mid, data_type)
            source_ang_bin = source_data[0]
            source_ang_bin = calc.ang_eff(source_ang_bin, echan, data_type)[0]
            source_occ = calc.src_occultation_bin(day, source_ra, source_dec, bin_time_mid, detector_name, data_type)[0]
            
            source_ang_bin[np.where(source_occ == 0)] = 0
            source_ang_bin[np.where(total_rate == 0)] = 0
            
            source_ang_bin[source_ang_bin>0] = source_ang_bin[source_ang_bin>0] - np.mean(source_ang_bin[source_ang_bin>0])
            
            source_ang_bin = np.array(source_ang_bin)
            
            sources_ang_bin = np.concatenate((sources_ang_bin, [source_ang_bin]), axis = 0)
            pre_coeffs.append(-1.)
    
    pre_coeffs = np.array(pre_coeffs)
    
    #print sources_ang_bin
    
    def wrapper_fit_function(x, number, *args):
        coeffs = args[::1]
        return fit_function(x, number, coeffs)
    
    def fit_function(x, number, coeffs):
        this_is_it = np.zeros(len(bin_time_mid))
        for i in range(0, number):
            this_is_it = np.add(this_is_it, coeffs[i]*sources_ang_bin[i])
        return this_is_it
    
    sigma = np.array((np.fabs(residuals) + 1)**(0.5))
    
    fit_results = optimization.curve_fit(lambda x, *args: wrapper_fit_function(x, number, *args), bin_time_mid, residuals, pre_coeffs, sigma, maxfev = 10000)
    
    sub_coeffs = fit_results[0]
    
    fit_curve = fit_function(bin_time_mid, number, sub_coeffs)
    sub_residuals = residuals - fit_curve
    
    sources_plots = np.multiply(sources_ang_bin.T, sub_coeffs).T
    
    return sub_residuals, fit_curve, sources_ang_bin, sources_plots, sub_coeffs

In [30]:
day = 150926
echan = 1
detector = 'n3'
sources = np.array([[135.529, -40.555], [186.657, -62.77], [255.986, -37.844], [256.45, -36.417], [270.275, -25.083], [270.382, -20.531], [274.006, -14.036], [288.82, 10.97], [308.107, 40.958]])
sources_names = np.array(['VELAX-1', 'GX301-2', '4U1700-377', 'GX349+2', 'GX5-1', 'GX9+1', 'GX17+2', 'GRS1915+105', 'CYGX-3'])

measurement = calc.curve_fit_plots(day, detector, echan)
residuals = measurement[0]
plot_time_bin = measurement[10]

data = subtract_sources(residuals, sources, detector, echan, day)
sub_residuals = data[0]
fit_curve = data[1]
sources_ang_bin = data[2]
sources_plots = data[3]
sub_coeffs = data[4]

In [31]:
mpl.rcParams['axes.color_cycle'] = ['b', 'g', 'c', 'm', 'y', 'k']

for i in range(0, len(sources)):
    plt.plot(plot_time_bin, sources_plots[i], label = sources_names[i])

#plt.plot(plot_time_bin, residuals, 'b-')
plt.plot(plot_time_bin, fit_curve, 'r-')
#plt.plot(plot_time_bin, sub_residuals, 'r-')

plt.grid()
plt.legend()
plt.show()

In [32]:
print sub_coeffs

[ 0.11805418  0.56242399  0.06765662  0.08894685 -1.23191628  0.52484047
  0.02512609 -0.01682369 -0.08285396]


In [50]:
sources_coeff = np.array([2., 4., 10.])

sources_ang_bin = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [54]:
np.multiply(sources_coeff,sources_ang_bin.T).T

array([[  2.,   4.,   6.],
       [ 16.,  20.,  24.],
       [ 70.,  80.,  90.]])

In [55]:
np.squeeze(np.sum(np.multiply(sources_coeff,sources_ang_bin.T).T, axis = 0))

array([  88.,  104.,  120.])

In [2]:
import numpy as np

In [16]:
a = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9],[10, 11, 12]])
b = np.array([[13, 14]])

a_list = a.tolist()
b_list = b.tolist()

c = a_list + b_list

print c

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14]]
